In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings(action='ignore')
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.feature_column as fc 
import tensorflow.debugging as tfdbg
from sklearn.model_selection import train_test_split

df = pd.read_csv("data/train.flat.csv", dtype={'fullVisitorId': 'str', 'visitId': 'str'})
df_t = pd.read_csv("data/test.flat.csv", dtype={'fullVisitorId': 'str', 'visitId': 'str'})
for col in df.columns:
#     print(col, df[col].fillna('').nunique())
    if df[col].fillna('').nunique() == 1:
        print("column skipped (cardinality = 1):", col)
        df = df.drop(col, axis=1)

column skipped (cardinality = 1): device.mobileDeviceInfo
column skipped (cardinality = 1): device.browserVersion
column skipped (cardinality = 1): totals.visits
column skipped (cardinality = 1): geoNetwork.networkLocation
column skipped (cardinality = 1): device.screenResolution
column skipped (cardinality = 1): geoNetwork.longitude
column skipped (cardinality = 1): geoNetwork.latitude
column skipped (cardinality = 1): device.flashVersion
column skipped (cardinality = 1): device.language
column skipped (cardinality = 1): trafficSource.adwordsClickInfo.criteriaParameters
column skipped (cardinality = 1): device.screenColors
column skipped (cardinality = 1): device.mobileDeviceModel
column skipped (cardinality = 1): socialEngagementType
column skipped (cardinality = 1): device.operatingSystemVersion
column skipped (cardinality = 1): device.mobileInputSelector
column skipped (cardinality = 1): device.mobileDeviceBranding
column skipped (cardinality = 1): geoNetwork.cityId
column skipped 

In [2]:
sdf = pd.DataFrame()
sdf_t = pd.DataFrame()
features = []

# boolean
for col in ['trafficSource.isTrueDirect', 'trafficSource.adwordsClickInfo.isVideoAd', 'device.isMobile', 'totals.newVisits']:
    sdf[col] = df[col].apply(lambda x: 2 if np.isnan(x) else int(x))
    sdf_t[col] = df_t[col].apply(lambda x: 2 if np.isnan(x) else int(x))
    features.append(fc.indicator_column(fc.categorical_column_with_vocabulary_list(col, (0,1,2))))
        
# numeric
for col in ['visitNumber', 'visitStartTime', 'date', 'totals.hits', 'totals.pageviews']:
    minv = df[col].min()
    maxv = df[col].max()
    sdf[col] = df[col].apply(lambda x: (x-minv)/maxv).fillna(0)
    sdf_t[col] = df_t[col].apply(lambda x: (x-minv)/maxv).fillna(0)
    features.append(fc.numeric_column(col))

    
from sklearn.preprocessing import LabelEncoder
    
# string
for col in [
    'trafficSource.adwordsClickInfo.adNetworkType',
    'trafficSource.adwordsClickInfo.page',
    'channelGrouping',
    'geoNetwork.region',
    'geoNetwork.country',
    'geoNetwork.metro',
    'trafficSource.campaign',
    'totals.bounces',
    #'trafficSource.referralPath',
    'device.deviceCategory',
    'device.browser',
    'trafficSource.adContent',
    'trafficSource.medium',
    'geoNetwork.city',
    'trafficSource.source',
    'trafficSource.adwordsClickInfo.slot',
    'fullVisitorId',
    'geoNetwork.subContinent',
    'device.operatingSystem',
    #'geoNetwork.networkDomain',
    'trafficSource.adwordsClickInfo.gclId',
#     'trafficSource.campaignCode', // not exist in test
    'trafficSource.keyword',
    'geoNetwork.continent']:
    le = LabelEncoder()
    le.fit(pd.concat([df[col].astype(str).fillna(''), df_t[col].astype(str).fillna('')]))
    sdf[col] = le.transform(df[col].astype(str).fillna(''))
    sdf_t[col] = le.transform(df_t[col].astype(str).fillna(''))

In [8]:
from sklearn.linear_model import SGDClassifier
from sklearn import svm
from sklearn.metrics import roc_curve, auc

label = pd.to_numeric(df['totals.transactionRevenue'].fillna(0)>0)

In [7]:
clf = SGDClassifier()
clf.fit(sdf, label)
# clf = svm.SVC()
# clf.fit(sdf, label)
print(clf.score(sdf, label))
pred = clf.predict(sdf)

t_label = pd.DataFrame()
t_label['true'] = label
t_label['pred'] = pred

t_label

0.9872373577025695


,true,pred
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
5,False,False
6,False,False
7,False,False
8,False,False
9,False,False


In [5]:
pred = clf.predict(sdf_t)
print(pred)
print(sum(list(map(int, pred))) ) 

[False False False ... False False False]
0
